<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title"><b>Misc exercises</b></span> by <a xmlns:cc="http://creativecommons.org/ns#" href="http://mate.unipv.it/gualandi" property="cc:attributionName" rel="cc:attributionURL">Stefano Gualandi</a> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License</a>.<br />Based on a work at <a xmlns:dct="http://purl.org/dc/terms/" href="https://github.com/mathcoding/opt4ds" rel="dct:source">https://github.com/mathcoding/opt4ds</a>.

**NOTE:** Execute the following script whenever running this script on a Google Colab.

In [ ]:
import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("glpk") or os.path.isfile("glpk")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq glpk-utils
    else:
        try:
            !conda install -c conda-forge glpk 
        except:
            pass

## 3. Writing ILP Models with Pyomo
In this notebook, we show how to use Pyomo to write general LPs problems.

### 3.1 Solving the Lego Planning Problem
As a first exercise, you have to solve the **Linear Programming (LP)** problem that we have written to model the Lego Planning problem (see the slides on KIRO):

$$
\begin{align}
\max \quad & 8 c + 11 t  \\
 \quad & 2c + 2t \leq 24 \\
& c + 2 t \leq 18\\
& c \geq 0\\
& t \geq 0
\end{align}
$$

You have to use Pyomo to define the **variables**, the **objective function**, and the **constraints**.

**EXERCISE 1:** Using the template for the *Steel Production Planning Problem* solve the LEGO instance defined above.

In [ ]:
# Write your own script

**EXERCISE 2:** Modify the previous script to solve the second version of the Lego Planning problem:

$$
\begin{align}
\max \quad & 8 c + 11 t + 15 s \\
 \quad & 2c + 2t +2s \leq 24 \\
& c + 2 t +3s \leq 18\\
& c \geq 0\\
& t \geq 0\\
& s \geq 0
\end{align}
$$

Basically, you have to add a third variable to the model, to modify the objective function, and the two constraints. Later, you call the solver again, check the status of the solver, and the solution values.

In [ ]:
# Write your own script

### 3.2 Random LPs
Let consider the following python function that generates a random LP instance.

Recall that an LP is completely defined by the cost vector $c$, the rhs vector $b$, and the coefficient matrix $A$:

$$
    z = \min \,\{ c x \mid Ax \geq b, x \geq 0 \,\}
$$

Look at the following function.

In [ ]:
from numpy import random
def RandomILP(n, m, seed=13):
    random.seed(seed)
    c = random.randint(1, 10, size=n)
    b = random.randint(1, 10, size=m)
    A = random.randint(1, 10, size=(m,n))
    
    return c, b, A
    
print(RandomILP(2,3))

Next, we write another function that takes as input the data of an LP instance, and it builds the LP model using the Pyomo syntax, solves the instances, check the return status of the solver, and if possible print to video an optimal solution.

In [ ]:
from pyomo.environ import *

def SolveILP(c, b, A):
    m = len(b)
    n = len(c)
    
    model = ConcreteModel()
    
    # Be careful: RangeSet starts at 1 and not a 0, as python list index
    model.I = RangeSet(n)
    model.J = RangeSet(m)
    
    model.x = Var(model.I, within=NonNegativeIntegers)
    
    model.obj = Objective(expr = sum(c[i-1]*model.x[i] for i in model.I))
#                          sense = maximize)
    
    model.vincoli = ConstraintList()
    for j in model.J:
        model.vincoli.add(expr = sum(A[j-1, i-1]*model.x[i] for i in model.I) >= b[j-1])
    
    # Solver path
    solvername='glpk'
    # WARNING: The following path depends on your file system (windows, unix, mac)
    #          On Colab you can omit the executable paramater
    solverpath_folder='/Users/gualandi/solvers/glpk/bin/' 
    solverpath_exe=solverpath_folder+'glpsol'

    sol = SolverFactory(solvername).solve(model, tee=True)
    #sol = SolverFactory(solvername, executable=solverpath_exe).solve(model, tee=False)
    
    for info in sol['Solver']:
        print(info)
        
    xbar = [model.x[i]() for i in model.I]
    
    return 'optimal', model.obj(), xbar

Below we show an example of calling this function.

In [ ]:
c, b, A = RandomILP(2, 3, 1717)
print(c,b,A)
print(SolveILP(c, b, A))

### 3.3 Nurse Scheduling
Consider the nurse scheduling problem on the lecture' slides.

Let us write the same model, but in addition we use a cost larger than 1 for the Friday, Saturday, and Sunday nights.

For the model description we refer to the slides used during the lecture.

In [ ]:
# Import the numerical python library https://numpy.org/
import numpy as np

def StaffScheduling():
    # Data
    d = 7
    w = 5
    demand = [5,6,7,3,3,2,2]
    cost = [1.0, 1.0, 1.01, 1.02, 1.03, 1.07, 1.05]

    # Using the numpy library create a matrix of dimension "d x d"
    A = np.zeros( (d,d) )
    # Set the matrix coefficient to 1 using the patter of the slide
    for i in range(d):
        for j in range(w):
            A[(i+j)%d, i] = 1
    
    # Declare the model
    model = ConcreteModel()
    
    model.Day = RangeSet(d)
    
    model.x = Var(model.Day, within = NonNegativeIntegers)
    
    model.obj = Objective(expr=sum(cost[i-1]*model.x[i] for i in model.Day))
    
    model.cover = ConstraintList()
    for day in model.Day:
        model.cover.add(expr=sum(A[day-1, i-1]*model.x[i] for i in model.Day) >= demand[day-1])
    
    # Let us inspect (debug) the intermediate LP file
    model.write("staff.lp")
   
    # Solver path
    solvername='glpk'
    # WARNING: The following path depends on your file system (windows, unix, mac)
    # On Colab you can omit the executable paramater
    solverpath_folder='/Users/gualandi/solvers/glpk/bin/' 
    solverpath_exe=solverpath_folder+'glpsol'

    # Solve the LP model
    sol = SolverFactory(solvername).solve(model, tee=True)
    #sol = SolverFactory(solvername, executable=solverpath_exe).solve(model, tee=True)
    
    # Check the status
    for info in sol['Solver']:
        print(info)
        
        
    # Look at the solution
    xbar = [model.x[i]() for i in model.Day]
    
    return 'optimal', model.obj(), xbar
 

In [ ]:
print(StaffScheduling())

In [ ]:
!more staff.lp

### 3.4 Exercise: Steel Recycle Blending Problem
The industrial steel can be easily recycled, since it is possible to burn any
scrap to get only liquid steel (without plastics, glasses, ...).
However, it is hard to separate each single metal presents in the scrap,
and as a consequence, beside iron, we can get also chromium, nichel, and
other impurities in the liquid steel.

Depending on the type of production, some metals are desirable, while others
are not. For example, the stainless steel 18/10 must have 18% of chromium and
10% of nichel (consider that chromium and nichel are very expensive, much more 
than the steel itself). 

**Problem Statement:** Suppose that the Rossi's Steel company of Voghera can choose to buy some iron
scrap block with different properties regarding the different metals contained in 
each block. The company want to produce at minimum cost 100 quintals of stainless 
steel 18/10, which must have at least 65% of iron and at most 1% of 
impurity materials. Which fraction of each block is going to buy?

The data of the problem are given below.

In [ ]:
# Data of the problem (in theory, read data from .csv or excel file)

# Blocks you can buy
Blocks = ['Block1','Block2','Block3','Block4','Block5','Block6']

Weights = [30, 90, 50, 70, 60, 50]  # In quintals
Costs = [50, 100, 80, 85, 92, 115]  # Thousand of euros

# Components of metal in each block (given in percentage)
Cs = [[93, 76, 74, 65, 72, 68],  # Ferro
      [5, 13, 11, 16, 6, 23],    # Cromo
      [0, 11, 12, 14, 20, 8],    # Nichel
      [2, 0, 3, 5, 2, 1]]        # Impurità

**EXERCISE 3:** First, write on paper a LP model to solve the steel production problem for the Rossi's Steel company

**EXERCISE 4:** Solve the LP using Pyomo and GLPK. 

### Exercise 3.5: The Magic Square Puzzle
The [Magic Square](https://en.wikipedia.org/wiki/Magic_square) puzzle asks to place into a grid of size $n \times n$ the digits from $1$ to $n^2$, in such a way that the sum of the digits in each row, the sum of digits in each column, and the sum of the digits on the two main diagonals, is equal to the same number.

You can play with a $4 \times 4$ puzzle on Google Sheet online at [magic square link](https://docs.google.com/spreadsheets/d/1OcicQdKbZXpSV4ooXsbGC2OFR5cSgwgWgUimdwcT0qA/edit?usp=sharing).

**EXERCISE 5:** Write an ILP model to solve the magic square puzzle of size $n$.

In [ ]:
# Write your model here